In [1]:
# https://www.relataly.com/crypto-market-cluster-analysis-using-affinity-propagation-python/8114/

In [2]:
# ! pip install cryptocmd
# ! pip install seaborn

In [3]:
from cryptocmd import CmcScraper
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
from sklearn import cluster, covariance, manifold
import requests
import json
import warnings
warnings.filterwarnings('ignore')

In [4]:
NUMBER_OF_CRYPTOCURRENCIES = 500
TIMEOUT = 10  # seconds

UPDATE_BASIC_DATA = False
UPDATE_HISTORICAL_DATA = False

# Create and Import Data 

In [5]:
def createBasicData():

    # get basic data
    print('Fetching basic data...')
    url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit={NUMBER_OF_CRYPTOCURRENCIES}&sortBy=market_cap&sortType=desc&convert=USD&cryptoType=all&tagType=all&audited=false'
    response = requests.get(url, timeout = TIMEOUT)
    data = json.loads(response.text)
    basicData = pd.DataFrame(data['data']['cryptoCurrencyList'])

    # clean basic data
    basicData = basicData[
        (basicData['isActive'] == 1) & 
        (basicData['dateAdded'].apply(lambda x: int(x[:4])) < 2021) & 
        (basicData['lastUpdated'].apply(lambda x: int(x[:4])) > 2022) &
        (basicData['tags'].apply(lambda x: 'stablecoin' not in x)) &  # TODO: Add Tether
        (basicData['marketPairCount'] > 5)
    ]

    # export basic data
    basicData.to_csv('./data/basic_data.csv')

    # return basic data
    print('Basic data fetched.')
    basicData.reset_index(drop = True, inplace = True)
    return basicData


def importBasicData():

    # import basic data
    basicData = pd.read_csv('./data/basic_data.csv')
    basicData = basicData.iloc[: , 1:]
    
    # return basic data
    basicData.reset_index(drop = True, inplace = True)
    return basicData

In [6]:
def createHistoricalData(basicData):

    # create symbols list and historical data dataframe
    symbols = basicData['symbol'].to_list()
    historicalData = pd.DataFrame()
    n = 0
    numberOfSymbols = len(symbols)

    # loop in symbols and get historical data
    for symbol in symbols:

        if ',' in symbol:
            symbol = symbol.split(',')[0]

        n += 1
        print(f'Fetching historical data for {symbol} ({n}/{numberOfSymbols})')
        scraper = CmcScraper(symbol)

        try:
            symbolHistoricalData = scraper.get_dataframe()
        except:
            print(f'Error in fetching historical data for {symbol}')
            continue

        symbolHistoricalData.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'marketcap']
        symbolHistoricalData.insert(0, 'symbol', symbol)
        symbolHistoricalData.insert(6, 'avg', (symbolHistoricalData['open'] + symbolHistoricalData['close']) / 2)
        symbolHistoricalData.insert(7, 'change', (symbolHistoricalData['close'] - symbolHistoricalData['open']) / symbolHistoricalData['open'])

        historicalData = pd.concat([historicalData, symbolHistoricalData])

    # export historical data
    historicalData.to_csv('./data/historical_data.csv')

    # return historical data
    print('Historical data fetched.')
    historicalData.reset_index(drop = True, inplace = True)
    return historicalData


def importHistoricalData():

    # import historical data
    historicalData = pd.read_csv('./data/historical_data.csv')
    historicalData = historicalData.iloc[: , 1:]

    # return historical data
    historicalData.reset_index(drop = True, inplace = True)
    return historicalData

In [7]:
if UPDATE_BASIC_DATA:
    basicData = createBasicData()
else:
    try:
        basicData = importBasicData()
    except FileNotFoundError:
        basicData = createBasicData()

if UPDATE_HISTORICAL_DATA:
    historicalData = createHistoricalData(basicData)
else:
    try:
        historicalData = importHistoricalData()
    except FileNotFoundError:
        historicalData = createHistoricalData(basicData)

# Clustering

In [8]:
def createChangesData(historicalData):

    changesData = historicalData[['date', 'symbol', 'change']]
    symbolsAge = changesData.groupby('symbol').count()['change']
    
    # keep symbols which have at least two years of changes data
    changesData = changesData[historicalData['symbol'].isin(dict(symbolsAge[symbolsAge >= 730]))]
    print(f'delete symbols which do not have at least two years of changes data:\n{[x for x in dict(symbolsAge[symbolsAge < 730]).keys()]}\n')

    # keep the last two years of changes data
    changesData = changesData.groupby('symbol').head(730)

    # keep symbols which have the last date of changes data
    symbolsLastDate = changesData.groupby('symbol').first()['date']
    lastDate = symbolsLastDate['BTC']
    changesData = changesData[historicalData['symbol'].isin(dict(symbolsLastDate[symbolsLastDate == lastDate]))]
    print(f'delete symbols which do not have the last day of changes data:\n{[x for x in dict(symbolsLastDate[symbolsLastDate != lastDate]).keys()]}\n')

    # keep symbols which have the first date of changes data
    symbolsFirstDate = changesData.groupby('symbol').last()['date']
    firstDate = symbolsFirstDate['BTC']
    changesData = changesData[historicalData['symbol'].isin(dict(symbolsFirstDate[symbolsFirstDate == firstDate]))]
    print(f'delete symbols which do not have the first day of changes data:\n{[x for x in dict(symbolsFirstDate[symbolsFirstDate != firstDate]).keys()]}\n')
    
    #create the pivot table of changes data
    changesData = changesData.pivot(index = 'date', columns = 'symbol')
    changesData = changesData['change']
    
    # return changes data
    return changesData

In [9]:
changesData = createChangesData(historicalData)
changesData

delete symbols which do not have at least two years of changes data:
['ACA', 'BTT', 'CFG', 'CORE', 'CQT', 'FCT', 'FLR', 'HIFI', 'TON', 'WEVER', 'WHBAR', 'WTRX']

delete symbols which do not have the last day of changes data:
[]

delete symbols which do not have the first day of changes data:
['AVINOC', 'CNX', 'LOCUS']



symbol,1INCH,AAVE,ABBC,ACH,ADA,ADS,AERGO,AGIX,ALGO,ALPHA,...,XYO,YFI,YFII,ZEC,ZEN,ZEON,ZIL,ZRX,ankrETH,stETH
date,,,,,,,,,,,,,,,,,,,,,
2021-05-15,-0.083220,-0.131665,-0.081620,-0.124162,0.083425,-0.150404,-0.064519,0.150867,-0.062463,-0.125322,...,-0.030908,-0.138942,-0.072608,-0.115543,-0.140801,-0.187051,-0.074110,-0.092368,-0.736920,-0.103279
2021-05-16,-0.023810,0.020371,-0.110980,0.008674,0.063404,0.058682,-0.002286,-0.061318,0.011243,-0.023672,...,-0.075491,-0.013425,0.017249,-0.027575,-0.014273,-0.139745,-0.014486,0.018487,-0.709512,0.034050
2021-05-17,-0.083917,0.089130,0.095968,-0.119050,-0.116497,-0.039780,-0.106534,-0.052754,-0.079952,-0.059007,...,-0.086203,-0.024438,-0.050844,-0.101355,-0.090599,0.073557,-0.073809,-0.064328,-0.083254,-0.130722
2021-05-18,0.124842,0.104940,-0.017899,0.070049,-0.012051,0.036617,0.105873,0.076460,0.064164,0.062784,...,0.162054,0.134930,0.061655,0.084522,0.006404,-0.220628,0.037117,0.092093,0.037580,0.030994
2021-05-19,-0.383851,-0.335111,-0.182394,-0.273651,-0.261171,-0.335110,0.098949,-0.311233,-0.309627,-0.388818,...,-0.211343,-0.370148,-0.423980,-0.417518,-0.363757,-0.885342,-0.361351,-0.371863,-0.267830,-0.259835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10,0.015339,0.004240,-0.004884,0.029610,0.016903,-0.010325,0.005931,0.056702,0.021871,0.039265,...,-0.011585,-0.037617,-0.048785,0.005139,0.022770,0.013812,0.021348,0.034374,-0.004780,0.000044
2023-05-11,-0.030726,-0.036844,-0.076328,-0.055683,-0.026808,0.005516,-0.048045,-0.071885,-0.035831,-0.073919,...,-0.034130,-0.054561,-0.033477,-0.023309,-0.031919,-0.017598,-0.045591,-0.050205,-0.019461,-0.023326
2023-05-12,0.013715,0.013205,-0.005927,0.012711,0.029062,-0.131866,0.000679,0.073661,0.011498,0.036063,...,0.039014,0.040854,0.010066,0.011033,0.035947,0.007814,0.013213,0.015130,0.004569,0.005416


In [10]:
changesData

symbol,1INCH,AAVE,ABBC,ACH,ADA,ADS,AERGO,AGIX,ALGO,ALPHA,...,XYO,YFI,YFII,ZEC,ZEN,ZEON,ZIL,ZRX,ankrETH,stETH
date,,,,,,,,,,,,,,,,,,,,,
2021-05-15,-0.083220,-0.131665,-0.081620,-0.124162,0.083425,-0.150404,-0.064519,0.150867,-0.062463,-0.125322,...,-0.030908,-0.138942,-0.072608,-0.115543,-0.140801,-0.187051,-0.074110,-0.092368,-0.736920,-0.103279
2021-05-16,-0.023810,0.020371,-0.110980,0.008674,0.063404,0.058682,-0.002286,-0.061318,0.011243,-0.023672,...,-0.075491,-0.013425,0.017249,-0.027575,-0.014273,-0.139745,-0.014486,0.018487,-0.709512,0.034050
2021-05-17,-0.083917,0.089130,0.095968,-0.119050,-0.116497,-0.039780,-0.106534,-0.052754,-0.079952,-0.059007,...,-0.086203,-0.024438,-0.050844,-0.101355,-0.090599,0.073557,-0.073809,-0.064328,-0.083254,-0.130722
2021-05-18,0.124842,0.104940,-0.017899,0.070049,-0.012051,0.036617,0.105873,0.076460,0.064164,0.062784,...,0.162054,0.134930,0.061655,0.084522,0.006404,-0.220628,0.037117,0.092093,0.037580,0.030994
2021-05-19,-0.383851,-0.335111,-0.182394,-0.273651,-0.261171,-0.335110,0.098949,-0.311233,-0.309627,-0.388818,...,-0.211343,-0.370148,-0.423980,-0.417518,-0.363757,-0.885342,-0.361351,-0.371863,-0.267830,-0.259835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10,0.015339,0.004240,-0.004884,0.029610,0.016903,-0.010325,0.005931,0.056702,0.021871,0.039265,...,-0.011585,-0.037617,-0.048785,0.005139,0.022770,0.013812,0.021348,0.034374,-0.004780,0.000044
2023-05-11,-0.030726,-0.036844,-0.076328,-0.055683,-0.026808,0.005516,-0.048045,-0.071885,-0.035831,-0.073919,...,-0.034130,-0.054561,-0.033477,-0.023309,-0.031919,-0.017598,-0.045591,-0.050205,-0.019461,-0.023326
2023-05-12,0.013715,0.013205,-0.005927,0.012711,0.029062,-0.131866,0.000679,0.073661,0.011498,0.036063,...,0.039014,0.040854,0.010066,0.011033,0.035947,0.007814,0.013213,0.015130,0.004569,0.005416
